In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Gold v1 — Features e Análises (IBOV, SP500, DXY)
# MAGIC
# MAGIC ## Objetivos
# MAGIC A partir das tabelas Silver, construir uma camada Gold orientada a análise quantitativa:
# MAGIC - Features técnicas do IBOV (retornos, médias, Bollinger, ATR, label)
# MAGIC - Retornos diários (IBOV, SP500, DXY) em base alinhada
# MAGIC - Correlação global (Pearson) e correlação dinâmica (rolling)
# MAGIC - Tabela analítica unificada para exploração e modelagem
# MAGIC
# MAGIC ## Entradas (Silver)
# MAGIC - `mvp_finance.silver_prices_long`
# MAGIC - `mvp_finance.silver_prices_wide_aligned` (preferencial)
# MAGIC - `mvp_finance.silver_returns_wide` (preferencial)
# MAGIC
# MAGIC ## Saídas (Gold)
# MAGIC - `mvp_finance.gold_ibov_features`
# MAGIC - `mvp_finance.gold_retornos_global`
# MAGIC - `mvp_finance.gold_corr_ibov_global`
# MAGIC - `mvp_finance.gold_analytics_ibov_global`


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## 0) Imports e contexto
# COMMAND ----------
import pyspark.sql.functions as F
from pyspark.sql.window import Window

spark.sql("USE mvp_finance")


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## 1) Leitura das tabelas Silver
# MAGIC Carregamos as tabelas e validamos rapidamente schemas/amostras.
# MAGIC
# MAGIC Observação:
# MAGIC - `wide_aligned` + `returns_wide` são preferíveis para evitar gaps e “matriz quebrada”.
# COMMAND ----------
silver_long   = spark.table("silver_prices_long")
silver_wide_a = spark.table("silver_prices_wide_aligned")
silver_ret    = spark.table("silver_returns_wide")

print("Schema silver_prices_long:")
silver_long.printSchema()

print("Schema silver_prices_wide_aligned:")
silver_wide_a.printSchema()

print("Schema silver_returns_wide:")
silver_ret.printSchema()

display(silver_long.limit(5))
display(silver_wide_a.limit(5))
display(silver_ret.limit(5))


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## 2) GOLD — Features técnicas Multi-Ativo (`gold_asset_features`)
# MAGIC
# MAGIC **Base:** `silver_prices_long` (todos os símbolos disponíveis)
# MAGIC
# MAGIC **Features (por símbolo):**
# MAGIC - `close_prev`, `retorno`, `retorno_log`
# MAGIC - `sma20`, `sma50`
# MAGIC - `boll_upper`, `boll_lower` (20 períodos, 2 desvios)
# MAGIC - `true_range`, `atr14` (ATR clássico via TR rolling)
# MAGIC - `label_direcao` (1 se retorno > 0, senão 0)
# MAGIC
# MAGIC **Observação de rigor:**
# MAGIC ATR usa:
# MAGIC `TR = max(high-low, |high-close_prev|, |low-close_prev|)`

# COMMAND ----------
import pyspark.sql.functions as F
from pyspark.sql.window import Window

# Janela por ativo (símbolo)
w_sym = Window.partitionBy("symbol").orderBy("timestamp")

features_df = (
    silver_long
    .orderBy("symbol", "timestamp")
    .withColumn("close_prev", F.lag("close", 1).over(w_sym))
    .withColumn("retorno", (F.col("close") / F.col("close_prev")) - F.lit(1))
    .withColumn("retorno_log", F.log(F.col("close") / F.col("close_prev")))
    .withColumn("sma20", F.avg("close").over(w_sym.rowsBetween(-19, 0)))
    .withColumn("sma50", F.avg("close").over(w_sym.rowsBetween(-49, 0)))
    .withColumn("std20", F.stddev("close").over(w_sym.rowsBetween(-19, 0)))
    .withColumn("boll_upper", F.col("sma20") + F.lit(2) * F.col("std20"))
    .withColumn("boll_lower", F.col("sma20") - F.lit(2) * F.col("std20"))
)

tr = F.greatest(
    (F.col("high") - F.col("low")),
    F.abs(F.col("high") - F.lag("close", 1).over(w_sym)),
    F.abs(F.col("low")  - F.lag("close", 1).over(w_sym)),
)

features_df = (
    features_df
    .withColumn("true_range", tr)
    .withColumn("atr14", F.avg("true_range").over(w_sym.rowsBetween(-13, 0)))
    .withColumn("label_direcao", F.when(F.col("retorno") > 0, F.lit(1)).otherwise(F.lit(0)))
)

display(features_df.select(
    "symbol","timestamp","date","open","high","low","close","volume",
    "retorno","retorno_log","sma20","sma50","boll_upper","boll_lower","atr14","label_direcao"
).orderBy("symbol","timestamp").limit(20))





## Análise Exploratória das Features — Exemplo com Bitcoin

Nesta seção realizamos uma **análise exploratória (EDA)** das features calculadas na camada **Gold**, com o objetivo de **validar visualmente** se os indicadores técnicos construídos fazem sentido do ponto de vista financeiro e estatístico.

### Por que escolhemos o Bitcoin?

O **Bitcoin foi escolhido de forma intencionalmente aleatória**, apenas como **ativo representativo** para demonstração visual.  
Os mesmos cálculos e gráficos **podem ser aplicados a qualquer outro ativo** presente no dataset (`NASDAQ`, `DOWJONES`, `PETROLEO`, `TREASURY10Y`, etc.), pois todas as features foram construídas de forma **genérica e multi-ativo**.

O uso do Bitcoin facilita a interpretação visual por apresentar:
- Alta volatilidade
- Regimes claros de tendência
- Movimentos amplos que tornam indicadores técnicos mais evidentes

##  Preço + Médias Móveis + Bandas de Bollinger

**Objetivo:** 

Validar se as médias móveis (`SMA20`, `SMA50`) e as Bandas de Bollinger capturam corretamente:
- Tendência
- Consolidação
- Regiões de sobrecompra e sobrevenda

**Interpretação esperada:**
- Preço acima da SMA20/SMA50 indica tendência de alta
- Toques ou rompimentos das bandas sugerem extremos de preço
- Estreitamento das bandas indica redução de volatilidade

Este gráfico confirma que as bandas e médias acompanham o comportamento do preço de forma coerente.

## ATR14 — Volatilidade

**Objetivo:** 

Analisar se o indicador de volatilidade (`ATR14`) reage adequadamente a períodos de estresse e calmaria do mercado.

**Interpretação esperada:**
- Picos de ATR coincidem com movimentos bruscos do preço
- ATR reduzido indica períodos de consolidação
- ATR serve como base para gestão de risco e dimensionamento de posição

O gráfico mostra claramente a alternância entre regimes de baixa e alta volatilidade.

## Retornos Diários

**Objetivo:**  

Verificar a sanidade estatística dos retornos:
- Presença de outliers
- Simetria em torno de zero
- Estabilidade ao longo do tempo

**Interpretação esperada:**
- Retornos oscilam em torno de zero
- Não há explosões artificiais (sanity check)
- Confirma que o cálculo de retorno está correto


## Distribuição dos Retornos

**Objetivo:**  

Avaliar a distribuição estatística dos retornos para futuras aplicações em:
- Modelagem estatística
- Machine Learning
- Gestão de risco

**Interpretação esperada:**
- Distribuição aproximadamente centrada em zero
- Caudas mais longas (característica comum de ativos financeiros)
- Não-normalidade esperada (fat tails)

Este gráfico confirma que os retornos não seguem uma distribuição perfeitamente normal, o que é consistente com mercados reais.

## Retorno vs Label de Direção

**Objetivo:** 

Validar se o `label_direcao` separa corretamente dias positivos e negativos.

**Definição do label:**
- `label_direcao = 1` → retorno positivo
- `label_direcao = 0` → retorno negativo ou zero

**Interpretação esperada:**
- Retornos associados ao label 1 tendem a ser positivos
- Retornos associados ao label 0 tendem a ser negativos
- Confirmação de que o label é consistente para uso em ML supervisionado

O boxplot mostra uma separação clara entre os grupos, validando a lógica do rótulo.

## Conclusão desta etapa

Esta análise exploratória confirma que:

- As **features técnicas estão corretamente calculadas**
- Os indicadores capturam padrões financeiros reais
- Os dados estão **prontos para uso em hipóteses, correlação, ML e geração de sinais**

A partir deste ponto, o pipeline está preparado para avançar para:
- Testes estatísticos formais
- Construção de scores determinísticos
- Modelos de Machine Learning
- Backtests e validação econômica


In [0]:
# COMMAND ----------
# GOLD — Plot: Preço + SMA + Bollinger (1 ativo)

import pandas as pd
import matplotlib.pyplot as plt

symbol = "BITCOIN"  # troque livremente

pdf = (
    features_df
    .filter(F.col("symbol") == symbol)
    .orderBy("timestamp")
    .select(
        "timestamp","close","sma20","sma50","boll_upper","boll_lower"
    )
    .dropna()
    .toPandas()
)

plt.figure(figsize=(14,6))
plt.plot(pdf["timestamp"], pdf["close"], label="Close", color="black")
plt.plot(pdf["timestamp"], pdf["sma20"], label="SMA20", linestyle="--")
plt.plot(pdf["timestamp"], pdf["sma50"], label="SMA50", linestyle="--")
plt.fill_between(
    pdf["timestamp"],
    pdf["boll_lower"],
    pdf["boll_upper"],
    alpha=0.2,
    label="Bollinger"
)

plt.title(f"{symbol} — Preço + Médias + Bollinger")
plt.legend()
plt.grid(True)
plt.show()


In [0]:
# COMMAND ----------
# GOLD — Plot: ATR14

pdf_atr = (
    features_df
    .filter(F.col("symbol") == symbol)
    .orderBy("timestamp")
    .select("timestamp","atr14")
    .dropna()
    .toPandas()
)

plt.figure(figsize=(14,4))
plt.plot(pdf_atr["timestamp"], pdf_atr["atr14"], color="darkred")
plt.title(f"{symbol} — ATR14 (Volatilidade)")
plt.grid(True)
plt.show()


In [0]:
# COMMAND ----------
# GOLD — Plot: Retornos

pdf_ret = (
    features_df
    .filter(F.col("symbol") == symbol)
    .orderBy("timestamp")
    .select("timestamp","retorno")
    .dropna()
    .toPandas()
)

plt.figure(figsize=(14,4))
plt.plot(pdf_ret["timestamp"], pdf_ret["retorno"], alpha=0.6)
plt.axhline(0, color="black", linewidth=1)
plt.title(f"{symbol} — Retornos Diários")
plt.grid(True)
plt.show()


In [0]:
# COMMAND ----------
# GOLD — Histograma de retornos

plt.figure(figsize=(6,4))
plt.hist(pdf_ret["retorno"], bins=100)
plt.title(f"{symbol} — Distribuição dos Retornos")
plt.grid(True)
plt.show()


In [0]:
# COMMAND ----------
# GOLD — Label vs Retorno

pdf_lbl = (
    features_df
    .filter(F.col("symbol") == symbol)
    .orderBy("timestamp")
    .select("retorno","label_direcao")
    .dropna()
    .toPandas()
)

pdf_lbl.boxplot(by="label_direcao", column="retorno", grid=True)
plt.title(f"{symbol} — Retorno vs Label Direção")
plt.suptitle("")
plt.show()


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ### 2.1) Persistência — `gold_asset_features`
# MAGIC Tabela única com features por símbolo, pronta para EDA e modelagem.
# COMMAND ----------
spark.sql("DROP TABLE IF EXISTS gold_asset_features")

features_df.write.format("delta").mode("overwrite").saveAsTable("gold_asset_features")

print("✅ Tabela criada: mvp_finance.gold_asset_features")
spark.sql("SELECT symbol, COUNT(*) AS qtde FROM gold_asset_features GROUP BY symbol ORDER BY qtde DESC").show()



In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## 3) GOLD — Retornos alinhados (multi-ativo) (`gold_retornos_global`)
# MAGIC
# MAGIC **Base:** `silver_returns_wide`
# MAGIC - contém `trade_date`, preços, colunas `*_prev` e retornos `*_ret`
# MAGIC - é a base canônica para correlação, heatmap e validação de hipóteses
# MAGIC
# MAGIC **Nota:**
# MAGIC Apesar do nome `gold_retornos_global`, a tabela passa a conter retornos **multi-ativo**.
# COMMAND ----------
import pyspark.sql.functions as F

# silver_ret == silver_returns_wide (já carregado antes)
df = silver_ret

if "trade_date" not in df.columns:
    raise RuntimeError("[GOLD RET] silver_returns_wide não possui a coluna trade_date.")

# colunas de preço (base): tudo que não é trade_date e não termina com _prev/_ret
price_cols = [
    c for c in df.columns
    if c != "trade_date" and not c.endswith("_prev") and not c.endswith("_ret")
]

prev_cols = [c for c in df.columns if c.endswith("_prev")]
ret_cols  = [c for c in df.columns if c.endswith("_ret")]

# ordena colunas para ficar estável
price_cols = sorted(price_cols)
prev_cols  = sorted(prev_cols)
ret_cols   = sorted(ret_cols)

gold_ret = (
    df.select("trade_date", *price_cols, *prev_cols, *ret_cols)
      .orderBy("trade_date")
)

display(gold_ret.limit(10))

# COMMAND ----------
spark.sql("DROP TABLE IF EXISTS gold_retornos_global")

gold_ret.write.format("delta").mode("overwrite").saveAsTable("gold_retornos_global")

print("✅ Tabela criada: mvp_finance.gold_retornos_global")
spark.sql("SELECT COUNT(*) AS qtde FROM gold_retornos_global").show()




In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ### 3.1) Quality Gate — `gold_retornos_global` (multi-ativo)
# MAGIC Validações:
# MAGIC - `trade_date` não nulo e único
# MAGIC - retornos `*_ret` finitos (sem NaN / sem explosões típicas de divisão por zero)
# MAGIC - limites conservadores para alertar outliers
# MAGIC
# MAGIC Observação:
# MAGIC - linhas iniciais podem ter NULL (por conta do `lag`), isso é normal.
# COMMAND ----------
import pyspark.sql.functions as F

# 1) trade_date não nulo
null_td = gold_ret.filter(F.col("trade_date").isNull()).count()
if null_td > 0:
    raise RuntimeError(f"[GOLD RET] trade_date NULL (qtde={null_td})")

# 2) trade_date único
dups_td = gold_ret.groupBy("trade_date").count().filter(F.col("count") > 1).count()
if dups_td > 0:
    raise RuntimeError(f"[GOLD RET] duplicidade por trade_date (qtde={dups_td})")

# 3) identifica colunas de retorno
ret_cols = [c for c in gold_ret.columns if c.endswith("_ret")]
if not ret_cols:
    raise RuntimeError("[GOLD RET] Nenhuma coluna *_ret encontrada para validação.")

# mantemos apenas linhas com pelo menos 1 retorno não-nulo (para não matar o dataset no início)
ret_clean = gold_ret.filter(
    F.greatest(*[F.col(c).isNotNull().cast("int") for c in ret_cols]) == 1
)

# 4) NaN check
nan_counts = (
    ret_clean
    .select(*[F.sum(F.isnan(F.col(c)).cast("int")).alias(c) for c in ret_cols])
    .collect()[0]
    .asDict()
)

total_nan = sum(nan_counts.values())
if total_nan > 0:
    print("[GOLD RET] NaNs detectados (por coluna):")
    for k, v in sorted(nan_counts.items(), key=lambda kv: kv[1], reverse=True):
        if v > 0:
            print(f"  - {k}: {v}")
    raise RuntimeError(f"[GOLD RET] Existem NaNs em retornos (total={total_nan}).")

# 5) “Inf-like” guard (retornos absurdamente altos geralmente são dado ruim)
INF_GUARD = 10.0  # 1000% (guarda técnica)
inf_like = ret_clean.filter(
    F.greatest(*[F.abs(F.col(c)) for c in ret_cols]) > F.lit(INF_GUARD)
)
inf_cnt = inf_like.count()
if inf_cnt > 0:
    display(inf_like.select("trade_date", *ret_cols).orderBy("trade_date"))
    raise RuntimeError(f"[GOLD RET] Retornos com |ret| > {INF_GUARD} (possível Inf/divisão por zero). qtde={inf_cnt}")

# 6) sanity check conservador
abs_limit = 0.50
bad = ret_clean.filter(
    F.greatest(*[F.abs(F.col(c)) for c in ret_cols]) > F.lit(abs_limit)
)

bad_cnt = bad.count()
if bad_cnt > 0:
    bad_ranked = bad.withColumn(
        "_max_abs_ret",
        F.greatest(*[F.abs(F.col(c)) for c in ret_cols])
    ).orderBy(F.col("_max_abs_ret").desc())

    display(bad_ranked.select("trade_date", "_max_abs_ret", *ret_cols).limit(200))
    raise RuntimeError(f"[GOLD RET] retornos fora do limite ±{abs_limit*100:.0f}% (qtde={bad_cnt})")

print("[GOLD RET] OK")



In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ### 3.2) Persistência — `gold_retornos_global`
# COMMAND ----------
spark.sql("DROP TABLE IF EXISTS gold_retornos_global")

gold_ret.write.format("delta").mode("overwrite").saveAsTable("gold_retornos_global")

print("✅ Tabela criada: mvp_finance.gold_retornos_global")
spark.sql("SELECT COUNT(*) AS qtde FROM gold_retornos_global").show()


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## 4) GOLD — Correlações (global e dinâmica) (multi-ativo)
# MAGIC
# MAGIC Produzimos:
# MAGIC - **Correlação global (Pearson)** no período para **todos os pares** de ativos (retornos)
# MAGIC - **Correlação rolling 21d** para **todos os pares** (evidência de mudança estrutural)
# MAGIC
# MAGIC Saídas (recomendado):
# MAGIC - `gold_corr_pairs_global` (1 linha por par no período inteiro)
# MAGIC - `gold_corr_pairs_rolling_21d` (série temporal por par)
# COMMAND ----------
import pyspark.sql.functions as F
from itertools import combinations
from pyspark.sql.window import Window

# ret_cols: todas as colunas *_ret presentes em gold_ret
ret_cols = [c for c in gold_ret.columns if c.endswith("_ret")]
if not ret_cols:
    raise RuntimeError("[GOLD CORR] Nenhuma coluna *_ret encontrada em gold_ret.")

# nomes "limpos" dos ativos (remove _ret)
assets = [c.replace("_ret", "") for c in ret_cols]

print("[GOLD CORR] ativos detectados:", assets)
# COMMAND ----------
# Gera expressões corr(col_i, col_j) para todos os pares
pair_exprs = []
pairs = list(combinations(ret_cols, 2))

for a, b in pairs:
    a_name = a.replace("_ret", "")
    b_name = b.replace("_ret", "")
    pair_exprs.append(F.corr(F.col(a), F.col(b)).alias(f"corr__{a_name}__{b_name}"))

# Uma linha com todas as correlações (wide)
corr_wide = gold_ret.select(*pair_exprs)

display(corr_wide)
# COMMAND ----------
# Traz para driver (pouquíssimo dado: só 1 linha)
row = corr_wide.collect()[0].asDict()

corr_rows = []
for k, v in row.items():
    # k = "corr__A__B"
    _, a, b = k.split("__")
    corr_rows.append((a, b, float(v) if v is not None else None))

corr_global_pairs = spark.createDataFrame(corr_rows, ["asset_a", "asset_b", "corr_pearson"])

display(corr_global_pairs.orderBy(F.col("corr_pearson").desc_nulls_last()))
# COMMAND ----------
spark.sql("DROP TABLE IF EXISTS gold_corr_pairs_global")
corr_global_pairs.write.format("delta").mode("overwrite").saveAsTable("gold_corr_pairs_global")
print("✅ Tabela criada: mvp_finance.gold_corr_pairs_global")




In [0]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F

w21 = Window.orderBy("trade_date").rowsBetween(-20, 0)

def rolling_corr_safe(df, x, y, out, min_points=10, eps=1e-12):
    """
    Rolling corr robusta:
    - exige min_points válidos na janela
    - evita divisão por zero (stddev=0)
    - clampa pequenas violações numéricas para [-1, 1]
    """
    xcol = F.col(x)
    ycol = F.col(y)

    # quantos pares (x,y) não-nulos na janela
    n = F.count(xcol * ycol).over(w21)

    ex  = F.avg(xcol).over(w21)
    ey  = F.avg(ycol).over(w21)
    exy = F.avg(xcol * ycol).over(w21)

    cov = exy - (ex * ey)
    sx  = F.stddev(xcol).over(w21)
    sy  = F.stddev(ycol).over(w21)

    denom = sx * sy
    raw = cov / denom

    # validade: pontos suficientes e denom > 0
    valid = (n >= F.lit(min_points)) & denom.isNotNull() & (denom > F.lit(0))

    # clamp para tolerar artefatos numéricos mínimos
    clamped = (
        F.when(raw > F.lit(1) + F.lit(eps), F.lit(1.0))
         .when(raw < F.lit(-1) - F.lit(eps), F.lit(-1.0))
         .otherwise(raw)
    )

    return df.withColumn(out, F.when(valid, clamped).otherwise(F.lit(None)))


In [0]:
from itertools import combinations

ret_cols = [c for c in gold_ret.columns if c.endswith("_ret")]
pairs = list(combinations(ret_cols, 2))

df_roll = (
    gold_ret
    .select("trade_date", *ret_cols)
    .orderBy("trade_date")
)

out_cols = []
for a, b in pairs:
    a_name = a.replace("_ret", "")
    b_name = b.replace("_ret", "")
    out = f"corr__{a_name}__{b_name}__21d"
    df_roll = rolling_corr_safe(df_roll, a, b, out, min_points=10)
    out_cols.append(out)

# long format: (trade_date, asset_a, asset_b, corr_21d)
expr_stack = []
for out in out_cols:
    _, a, b, _ = out.split("__")
    expr_stack.append(
        F.struct(
            F.lit(a).alias("asset_a"),
            F.lit(b).alias("asset_b"),
            F.col(out).alias("corr_21d")
        )
    )

corr_roll_long = (
    df_roll
    .select("trade_date", F.explode(F.array(*expr_stack)).alias("x"))
    .select("trade_date", "x.asset_a", "x.asset_b", "x.corr_21d")
)

display(corr_roll_long.orderBy("trade_date").limit(50))


In [0]:
# tolerância numérica
TOL = 1e-6

bad_corr = (
    corr_roll_long
    .dropna(subset=["corr_21d"])
    .filter((F.col("corr_21d") > F.lit(1 + TOL)) | (F.col("corr_21d") < F.lit(-1 - TOL)))
    .count()
)

if bad_corr > 0:
    display(
        corr_roll_long
        .dropna(subset=["corr_21d"])
        .filter((F.col("corr_21d") > F.lit(1 + TOL)) | (F.col("corr_21d") < F.lit(-1 - TOL)))
        .orderBy("trade_date")
    )
    raise RuntimeError(f"[GOLD CORR ROLL] valores fora de [-1,1] além da tolerância (qtde={bad_corr})")

print("[GOLD CORR ROLL] OK — robusto com clamp/guard")

spark.sql("DROP TABLE IF EXISTS mvp_finance.gold_corr_pairs_rolling_21d")
corr_roll_long.write.format("delta").mode("overwrite").saveAsTable("mvp_finance.gold_corr_pairs_rolling_21d")
print("✅ Tabela criada: mvp_finance.gold_corr_pairs_rolling_21d")



In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ### 4.1) Correlação dinâmica (rolling 21d) — multi-ativo
# MAGIC Implementação via janelas (fórmula rolling):
# MAGIC
# MAGIC corr(X,Y) = cov(X,Y) / (std(X)*std(Y))
# MAGIC cov = E[XY] - E[X]E[Y]
# MAGIC
# MAGIC **Janela:** 21 pregões (ajustável).
# MAGIC
# MAGIC **Saída (recomendada):**
# MAGIC - `gold_corr_pairs_rolling_21d` (formato longo: trade_date + par + correlação)
# COMMAND ----------
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from itertools import combinations

# Janela rolling: 21 (inclui o dia atual)
w21 = Window.orderBy("trade_date").rowsBetween(-20, 0)

def rolling_corr(df, x, y, out):
    ex  = F.avg(F.col(x)).over(w21)
    ey  = F.avg(F.col(y)).over(w21)
    exy = F.avg((F.col(x) * F.col(y))).over(w21)
    cov = exy - (ex * ey)
    sx  = F.stddev(F.col(x)).over(w21)
    sy  = F.stddev(F.col(y)).over(w21)
    return df.withColumn(out, cov / (sx * sy))

# detecta retornos disponíveis
ret_cols = [c for c in gold_ret.columns if c.endswith("_ret")]
if not ret_cols:
    raise RuntimeError("[GOLD CORR ROLL] Nenhuma coluna *_ret encontrada em gold_ret.")

pairs = list(combinations(ret_cols, 2))

# base mínima para rolling
df_roll = gold_ret.select("trade_date", *ret_cols).orderBy("trade_date")

# calcula rolling para todos os pares, gerando colunas corr__A__B__21d
out_cols = []
for a, b in pairs:
    a_name = a.replace("_ret", "")
    b_name = b.replace("_ret", "")
    out = f"corr__{a_name}__{b_name}__21d"
    df_roll = rolling_corr(df_roll, a, b, out)
    out_cols.append(out)

# converte para formato longo: (trade_date, asset_a, asset_b, corr_21d)
stack_structs = []
for out in out_cols:
    # out = corr__A__B__21d
    _, a_name, b_name, _ = out.split("__")
    stack_structs.append(
        F.struct(
            F.lit(a_name).alias("asset_a"),
            F.lit(b_name).alias("asset_b"),
            F.col(out).alias("corr_21d")
        )
    )

corr_roll_long = (
    df_roll
    .select("trade_date", F.explode(F.array(*stack_structs)).alias("x"))
    .select("trade_date", "x.asset_a", "x.asset_b", "x.corr_21d")
)

display(corr_roll_long.orderBy("trade_date").limit(30))


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC #### Quality Gate — rolling corr em [-1, 1] quando não nula
# COMMAND ----------
from pyspark.sql import Window
import pyspark.sql.functions as F

w21 = Window.orderBy("trade_date").rowsBetween(-20, 0)

def rolling_corr_safe(df, x, y, out, min_points=10, eps=1e-9):
    """
    Rolling corr robusta:
    - exige um mínimo de pontos válidos (min_points) para calcular
    - evita divisão por zero (stddev = 0 ou NULL)
    - clampa pequenos overshoots numéricos em [-1,1]
    """
    cx  = F.count(F.col(x)).over(w21)
    cy  = F.count(F.col(y)).over(w21)
    cxy = F.count(F.col(x) * F.col(y)).over(w21)  # conta linhas onde ambos existem
    ex  = F.avg(F.col(x)).over(w21)
    ey  = F.avg(F.col(y)).over(w21)
    exy = F.avg(F.col(x) * F.col(y)).over(w21)
    cov = exy - (ex * ey)
    sx  = F.stddev(F.col(x)).over(w21)
    sy  = F.stddev(F.col(y)).over(w21)

    denom = sx * sy
    raw = cov / denom

    # condições de validade
    valid = (
        (cxy >= F.lit(min_points)) &
        denom.isNotNull() &
        (denom > F.lit(0))
    )

    # clamp numérico: aceita pequeno overshoot por eps
    clamped = (
        F.when(raw > F.lit(1) + F.lit(eps), F.lit(1))
         .when(raw < F.lit(-1) - F.lit(eps), F.lit(-1))
         .otherwise(raw)
    )

    return df.withColumn(out, F.when(valid, clamped).otherwise(F.lit(None)))
    from itertools import combinations

    ret_cols = [c for c in gold_ret.columns if c.endswith("_ret")]
    pairs = list(combinations(ret_cols, 2))
    df_roll = gold_ret.select("trade_date", *ret_cols).orderBy("trade_date")
    out_cols = []
    for a, b in pairs:
        a_name = a.replace("_ret", "")
        b_name = b.replace("_ret", "")
        out = f"corr__{a_name}__{b_name}__21d"
        df_roll = rolling_corr_safe(df_roll, a, b, out, min_points=10, eps=1e-9)
        out_cols.append(out)
    stack_structs = []
    for out in out_cols:
        _, a_name, b_name, _ = out.split("__")
        stack_structs.append(
            F.struct(
                F.lit(a_name).alias("asset_a"),
                F.lit(b_name).alias("asset_b"),
                F.col(out).alias("corr_21d")
            )
        )

    corr_roll_long = (
        df_roll
        .select("trade_date", F.explode(F.array(*stack_structs)).alias("x"))
        .select("trade_date", "x.asset_a", "x.asset_b", "x.corr_21d")
    )

    display(corr_roll_long.orderBy("trade_date").limit(30))
    tol = 1e-6
    bad_corr = (
        corr_roll_long
        .dropna(subset=["corr_21d"])
        .filter((F.col("corr_21d") > F.lit(1 + tol)) | (F.col("corr_21d") < F.lit(-1 - tol)))
        .count()
    )

    if bad_corr > 0:
        display(
            corr_roll_long
            .dropna(subset=["corr_21d"])
            .filter((F.col("corr_21d") > F.lit(1 + tol)) | (F.col("corr_21d") < F.lit(-1 - tol)))
            .orderBy("trade_date")
            .limit(200)
        )
        raise RuntimeError(f"[GOLD CORR ROLL] valores fora de [-1,1] além da tolerância (qtde={bad_corr})")

    print("[GOLD CORR ROLL] OK")

    spark.sql("DROP TABLE IF EXISTS gold_corr_pairs_rolling_21d")
    corr_roll_long.write.format("delta").mode("overwrite").saveAsTable("gold_corr_pairs_rolling_21d")
    print("✅ Tabela criada: mvp_finance.gold_corr_pairs_rolling_21d")



In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## Heatmap — Matriz de correlação global (retornos, multi-ativo)
# MAGIC
# MAGIC - Fonte: `gold_corr_pairs_global`
# MAGIC - Correlação de Pearson no período inteiro
# MAGIC - Funciona automaticamente para N ativos
# COMMAND ----------
import pandas as pd
import plotly.express as px

# lê da Gold (fonte oficial)
corr_pairs_pd = (
    spark.table("gold_corr_pairs_global")
    .toPandas()
)

# lista completa de ativos
assets = sorted(
    set(corr_pairs_pd["asset_a"]).union(corr_pairs_pd["asset_b"])
)

# inicializa matriz identidade
corr_matrix = pd.DataFrame(
    data=1.0,
    index=assets,
    columns=assets
)

# preenche pares
for _, row in corr_pairs_pd.iterrows():
    a, b, v = row["asset_a"], row["asset_b"], row["corr_pearson"]
    corr_matrix.loc[a, b] = v
    corr_matrix.loc[b, a] = v  # simetria

# heatmap
fig = px.imshow(
    corr_matrix,
    text_auto=".2f",
    aspect="auto",
    zmin=-1, zmax=1,
    color_continuous_scale="RdBu_r",
    title="Matriz de Correlação Global — Retornos (Pearson)"
)

fig.update_layout(
    xaxis_title="Ativo",
    yaxis_title="Ativo"
)

fig.show()



In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ## 5) GOLD — Tabela analítica unificada (multi-ativo) (`gold_analytics_assets`)
# MAGIC
# MAGIC Junta (por **ativo** e **dia**):
# MAGIC - Features técnicas do ativo (Gold features)
# MAGIC - Retornos/preços wide (contexto multi-ativo)
# MAGIC - Correlações rolling 21d do ativo vs demais
# MAGIC
# MAGIC Resultado: dataset pronto para EDA, validação de hipóteses e ML.
# COMMAND ----------
spark.sql("USE mvp_finance")

import pyspark.sql.functions as F
from pyspark.sql.window import Window
from itertools import combinations

def table_exists(db: str, name: str) -> bool:
    try:
        spark.table(f"{db}.{name}")
        return True
    except Exception:
        return False

DB = "mvp_finance"

# possíveis nomes (caso você tenha persistido com nomes antigos)
CANDIDATES = [
    "gold_corr_pairs_rolling_21d",
    "gold_corr_ibov_global_rolling",
    "gold_corr_pairs_rolling_21d".lower(),
    "gold_corr_ibov_global_rolling".lower(),
]

rolling_table = next((t for t in CANDIDATES if table_exists(DB, t)), None)

gold_ret = spark.table(f"{DB}.gold_retornos_global").withColumnRenamed("trade_date", "trade_date")  # só garantindo referência

if rolling_table:
    print(f"[OK] Usando rolling existente: {DB}.{rolling_table}")
    corr_roll_long = (
        spark.table(f"{DB}.{rolling_table}")
        .select("trade_date", "asset_a", "asset_b", "corr_21d")
    )
else:
    print("[INFO] Rolling não encontrado em tabela. Recalculando a partir de gold_retornos_global...")

    # detecta todas colunas *_ret
    ret_cols = [c for c in gold_ret.columns if c.endswith("_ret")]
    if not ret_cols:
        raise RuntimeError("[ROLLING] gold_retornos_global não possui colunas *_ret.")

    pairs = list(combinations(ret_cols, 2))

    w21 = Window.orderBy("trade_date").rowsBetween(-20, 0)

    def rolling_corr_safe(df, x, y, out, min_points=10, eps=1e-9):
        cxy = F.count(F.col(x) * F.col(y)).over(w21)
        ex  = F.avg(F.col(x)).over(w21)
        ey  = F.avg(F.col(y)).over(w21)
        exy = F.avg(F.col(x) * F.col(y)).over(w21)
        cov = exy - (ex * ey)
        sx  = F.stddev(F.col(x)).over(w21)
        sy  = F.stddev(F.col(y)).over(w21)
        denom = sx * sy
        raw = cov / denom

        valid = (cxy >= F.lit(min_points)) & denom.isNotNull() & (denom > F.lit(0))

        clamped = (
            F.when(raw > F.lit(1) + F.lit(eps), F.lit(1))
             .when(raw < F.lit(-1) - F.lit(eps), F.lit(-1))
             .otherwise(raw)
        )
        return df.withColumn(out, F.when(valid, clamped).otherwise(F.lit(None)))

    df_roll = gold_ret.select("trade_date", *ret_cols).orderBy("trade_date")

    out_cols = []
    for a, b in pairs:
        a_name = a.replace("_ret", "")
        b_name = b.replace("_ret", "")
        out = f"corr__{a_name}__{b_name}__21d"
        df_roll = rolling_corr_safe(df_roll, a, b, out)
        out_cols.append(out)

    # formato longo
    stack_structs = []
    for out in out_cols:
        _, a_name, b_name, _ = out.split("__")
        stack_structs.append(
            F.struct(
                F.lit(a_name).alias("asset_a"),
                F.lit(b_name).alias("asset_b"),
                F.col(out).alias("corr_21d")
            )
        )

    corr_roll_long = (
        df_roll
        .select("trade_date", F.explode(F.array(*stack_structs)).alias("x"))
        .select("trade_date", "x.asset_a", "x.asset_b", "x.corr_21d")
    )

    # persiste (opcional, mas recomendado)
    spark.sql(f"DROP TABLE IF EXISTS {DB}.gold_corr_pairs_rolling_21d")
    corr_roll_long.write.format("delta").mode("overwrite").saveAsTable(f"{DB}.gold_corr_pairs_rolling_21d")
    print(f"✅ Tabela criada: {DB}.gold_corr_pairs_rolling_21d")

gold_features = spark.table("gold_asset_features")
gold_ret_wide  = spark.table("gold_retornos_global")
# corr_roll_long já está definido pelo resolver acima
ret_small = gold_ret_wide.withColumnRenamed("trade_date", "date")

corr_sym = (
    corr_roll_long
    .withColumnRenamed("trade_date", "date")
)


In [0]:
import pyspark.sql.functions as F

spark.sql("USE mvp_finance")

# =========================
# 1) Fontes Gold existentes
# =========================
gold_features = spark.table("gold_asset_features")              # por symbol + date
gold_ret      = spark.table("gold_retornos_global")             # contexto global (wide)
corr_roll     = spark.table("gold_corr_pairs_rolling_21d")      # long: date, asset_a, asset_b, corr_21d

# =========================
# 2) Retornos (padroniza chave)
# =========================
ret_small = (
    gold_ret
    .withColumnRenamed("trade_date", "date")
)

# =========================
# 3) Correlação rolling → pivot por (date, symbol)
# =========================
corr_sym = (
    corr_roll
    .withColumnRenamed("trade_date", "date")
)

corr_as_a = corr_sym.select(
    "date",
    F.col("asset_a").alias("symbol"),
    F.col("asset_b").alias("other"),
    "corr_21d"
)

corr_as_b = corr_sym.select(
    "date",
    F.col("asset_b").alias("symbol"),
    F.col("asset_a").alias("other"),
    "corr_21d"
)

corr_long = corr_as_a.unionByName(corr_as_b)

corr_pivot = (
    corr_long
    .groupBy("date", "symbol")
    .pivot("other")
    .agg(F.first("corr_21d"))
)

# renomeia colunas de correlação
for c in corr_pivot.columns:
    if c not in ("date", "symbol"):
        corr_pivot = corr_pivot.withColumnRenamed(c, f"corr_21d__{c}")

# =========================
# 4) Features por ativo
# =========================
feat_small = gold_features.select(
    "symbol",
    "date",
    "timestamp",
    "open","high","low","close","volume",
    "retorno","retorno_log",
    "sma20","sma50",
    "boll_upper","boll_lower",
    "atr14",
    "label_direcao"
)

# =========================
# 5) JOIN FINAL
# =========================
gold_analytics_assets = (
    feat_small.alias("f")
    .join(ret_small.alias("r"), ["date"], "inner")
    .join(corr_pivot.alias("c"), ["date", "symbol"], "left")
)

display(gold_analytics_assets.orderBy("date", "symbol").limit(20))

# =========================
# 6) Persistência DEFINITIVA
# =========================
spark.sql("DROP TABLE IF EXISTS mvp_finance.gold_analytics_assets")

(
    gold_analytics_assets
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("mvp_finance.gold_analytics_assets")
)

print("✅ Tabela criada: mvp_finance.gold_analytics_assets")


In [0]:
gold_final = spark.table("mvp_finance.gold_analytics_assets")
gold_final.count()
gold_final.display()


In [0]:
# date e symbol não nulos
assert gold_final.filter(F.col("date").isNull()).count() == 0
assert gold_final.filter(F.col("symbol").isNull()).count() == 0

# unicidade
assert (
    gold_final.groupBy("date","symbol").count().filter("count > 1").count()
) == 0

print("[GOLD FINAL] OK — multi-ativo consistente")


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ### 5.1) Quality Gate — `gold_analytics_assets` (multi-ativo) [UPGRADED]
# MAGIC Validações:
# MAGIC - `date` e `symbol` não nulos
# MAGIC - sem duplicidade por (date, symbol)
# MAGIC - colunas críticas existem
# MAGIC - checks numéricos básicos (close/volume/label)
# MAGIC - NULLs esperados apenas no aquecimento (por ativo)
# COMMAND ----------
import pyspark.sql.functions as F
from pyspark.sql.window import Window

DB = "mvp_finance"
TBL = f"{DB}.gold_analytics_assets"

gold_final = spark.table(TBL)

# 0) Normaliza date (garante que é DATE para ordenação e joins)
# Se já for DATE, to_date mantém igual.
gold_final = gold_final.withColumn("date", F.to_date(F.col("date")))

# 1) date e symbol não nulos
null_date = gold_final.filter(F.col("date").isNull()).count()
if null_date > 0:
    raise RuntimeError(f"[GOLD FINAL] date NULL (qtde={null_date})")

null_symbol = gold_final.filter(F.col("symbol").isNull()).count()
if null_symbol > 0:
    raise RuntimeError(f"[GOLD FINAL] symbol NULL (qtde={null_symbol})")

# 2) unicidade por (date, symbol)
dups_key = (
    gold_final
    .groupBy("date", "symbol")
    .count()
    .filter(F.col("count") > 1)
    .count()
)
if dups_key > 0:
    display(
        gold_final.groupBy("date","symbol").count().filter(F.col("count") > 1)
                 .orderBy(F.col("count").desc(), "date", "symbol")
                 .limit(200)
    )
    raise RuntimeError(f"[GOLD FINAL] duplicidade por (date, symbol) (qtde={dups_key})")

# 3) colunas obrigatórias (mínimo de contrato)
required_cols = [
    "date", "symbol",
    "close", "retorno", "label_direcao"
]
missing = [c for c in required_cols if c not in gold_final.columns]
if missing:
    raise RuntimeError(f"[GOLD FINAL] colunas obrigatórias ausentes: {missing}")

# 4) sanity checks numéricos básicos
# - close deve ser > 0 (para índices/ativos típicos; se houver algum que possa ser 0, ajuste)
bad_close = gold_final.filter(F.col("close").isNull() | (F.col("close") <= 0)).count()
if bad_close > 0:
    display(gold_final.filter(F.col("close").isNull() | (F.col("close") <= 0))
                 .select("date","symbol","close").orderBy("date","symbol").limit(200))
    raise RuntimeError(f"[GOLD FINAL] close inválido (NULL ou <=0) (qtde={bad_close})")

# - volume deve ser >= 0 (se algum ativo não tiver volume, pode ficar NULL — aqui não forçamos)
if "volume" in gold_final.columns:
    bad_vol = gold_final.filter(F.col("volume").isNotNull() & (F.col("volume") < 0)).count()
    if bad_vol > 0:
        display(gold_final.filter(F.col("volume").isNotNull() & (F.col("volume") < 0))
                     .select("date","symbol","volume").orderBy("date","symbol").limit(200))
        raise RuntimeError(f"[GOLD FINAL] volume negativo (qtde={bad_vol})")

# - label deve ser 0/1
bad_label = gold_final.filter(~F.col("label_direcao").isin(0, 1)).count()
if bad_label > 0:
    display(gold_final.filter(~F.col("label_direcao").isin(0,1))
                 .select("date","symbol","retorno","label_direcao").orderBy("date","symbol").limit(200))
    raise RuntimeError(f"[GOLD FINAL] label_direcao fora de {{0,1}} (qtde={bad_label})")

# - retorno não pode ser NaN/Inf quando não nulo
bad_ret = gold_final.filter(
    F.col("retorno").isNotNull() & (F.isnan("retorno") | F.col("retorno").isin(float("inf"), float("-inf")))
).count()
if bad_ret > 0:
    display(gold_final.filter(
        F.col("retorno").isNotNull() & (F.isnan("retorno") | F.col("retorno").isin(float("inf"), float("-inf")))
    ).select("date","symbol","close","retorno").orderBy("date","symbol").limit(200))
    raise RuntimeError(f"[GOLD FINAL] retorno NaN/Inf (qtde={bad_ret})")

# 5) aquecimento por ativo (depois disso esperamos retorno e label presentes)
W = Window.partitionBy("symbol").orderBy("date")
N_WARMUP = 60  # cobre SMA50 + Boll20 + ATR14 com folga

qc = (
    gold_final
    .withColumn("_rn", F.row_number().over(W))
    .filter(F.col("_rn") > F.lit(N_WARMUP))
)

critical_after = ["retorno", "label_direcao"]

null_post = qc.select(
    *[F.sum(F.col(c).isNull().cast("int")).alias(f"null_{c}") for c in critical_after]
).collect()[0].asDict()

bad_total = int(sum(null_post.values()))
if bad_total > 0:
    print("[GOLD FINAL] NULLs pós-aquecimento:", null_post)
    display(
        qc.filter(F.col("retorno").isNull() | F.col("label_direcao").isNull())
          .select("date","symbol","close","retorno","label_direcao")
          .orderBy("date","symbol")
          .limit(200)
    )
    raise RuntimeError(f"[GOLD FINAL] NULLs inesperados pós-aquecimento (total={bad_total})")

print("[GOLD FINAL] OK ✅")


In [0]:
# COMMAND ----------
# MAGIC %md
# MAGIC ### 5.2) Persistência — `gold_analytics_ibov_global`
# COMMAND ----------
spark.sql("DROP TABLE IF EXISTS gold_analytics_ibov_global")

gold_final.write.format("delta").mode("overwrite").saveAsTable("gold_analytics_ibov_global")

print("✅ Tabela criada: mvp_finance.gold_analytics_ibov_global")
spark.sql("SELECT COUNT(*) AS qtde FROM gold_analytics_ibov_global").show()
spark.sql("SELECT * FROM gold_analytics_ibov_global ORDER BY date LIMIT 10").show()


## Conclusão — Camada Gold

A camada **Gold** representa a **materialização analítica final** do pipeline, transformando dados padronizados da Silver em **informação pronta para decisão, pesquisa e modelagem preditiva**.

### Síntese do que foi construído
Nesta camada foram consolidados, de forma sistemática e escalável:

- **Features técnicas por ativo**, calculadas de forma independente e causal:

    - retornos simples e logarítmicos
    - médias móveis (SMA 20 / 50)
    - Bandas de Bollinger
    - True Range e ATR
    - labels direcionais para classificação

- **Retornos globais multi-ativo**, baseados em séries estacionárias, garantindo:

    - comparabilidade entre mercados
    - robustez estatística
    - adequação para correlação e Machine Learning

- **Correlação global (Pearson)** entre todos os pares de ativos, fornecendo:

    - visão estrutural do relacionamento entre mercados
    - base para seleção e redução de variáveis

- **Correlação dinâmica (rolling 21 períodos)**, permitindo:

    - identificação de mudanças de regime
    - análise temporal da força e direção das relações
    - diferenciação entre correlação estrutural e circunstancial

- **Dataset analítico multi-ativo**, integrando:

    - features técnicas do ativo
    - contexto global diário
    - correlações dinâmicas do ativo versus os demais

### Qualidade, rigor e governança

A camada Gold foi construída com **preocupação explícita com rigor estatístico**, incluindo:

- uso exclusivo de **retornos** para correlação (evitando não-estacionariedade)
- proteção contra divisões por zero e variância nula em janelas rolling
- Quality Gates para:
- chaves temporais
- unicidade por `(date, symbol)`
- limites matemáticos das correlações

Valores ausentes são tratados como **informação válida** apenas:

- durante o aquecimento inicial das janelas
- quando a estatística não é definida (ex.: variância zero)

### Resultado final
Ao final da camada Gold, o pipeline entrega um **ativo de dados de alto valor**, que:

- está pronto para **EDA avançada**
- suporta **validação formal de hipóteses**
- pode ser consumido diretamente por modelos de **Machine Learning**
- permite evolução para **sinais operacionais e backtests**

Esta camada encerra o ciclo de transformação de dados e inaugura o ciclo de **inteligência e tomada de decisão**, mantendo separação clara entre:
    - engenharia de dados (Bronze / Silver)
    - análise, modelagem e estratégia (Gold)

O pipeline, como um todo, encontra-se **coerente, escalável e cientificamente sólido**, pronto para uso em ambientes de pesquisa aplicada ou produção analítica.
